In [1]:
import numpy as np
import pandas as pd
import datetime
import json
import geojson
import math
import matplotlib as plt
import os
import sys

# import random trajectory generator
traj_path = "/Users/jimmy.shao/GPS_traj_random_gen/"
sys.path.append(os.path.abspath(traj_path))
from GPTG import GPTG


KeyboardInterrupt: 

In [2]:
# general settings
avg_v = 15 # m/s, agv speed
r = 100 # meter, pie radius, define how close is "near spot"
theta = np.pi/12 #radian, pie angle




In [3]:
# read pattern from 
from_col = ['A1', 'A5', 'B1', 'B2', 'B2a', 'B3_1', 'B3_2', 'B3_3', 'B3_4', 'B3_5', 'B3_6', 'B3_7', 'B3_8', 'B3_9', 'B3_10', 'B3_11', 'B4', 'B4a', 'B5', 'B7', 'E8a', 'E8b']
tw_taxi = pd.read_csv("tw_taxi_info.csv", usecols=from_col)
to_col = ['full_time', 'single_driver', 'travel_distance', 'off_day', 'off_day1', 'slot1', 'slot2', 'slot3', 'slot4', 'slot5', 'slot6', 'slot7', 'slot8', 'slot9', 'slot10', 'slot11', 'op_time', 'op_time1', 'no_passenger_time', 'trip', 'city', 'city1']
tw_taxi.columns = to_col
print(len(from_col))
print(len(to_col))

# accept full-time, single driver tp/ntp only
tw_taxi = tw_taxi[(tw_taxi['full_time'] == 1) & (tw_taxi['single_driver'] == 2) & (tw_taxi['city'] < 3) & (tw_taxi['city1'] < 3)]

22
22


In [4]:
# off day function
def off_day_transform(df):
    dic = {1: 0, 2:np.random.random_integers(1,2), 3:np.random.random_integers(3,4), 4:np.random.random_integers(5,6), 5:np.random.random_integers(7,8)}
    if df['off_day'] < 6:
        return dic[df['off_day']]
    else:
        return df['off_day1']
    
# operation duration function
def op_time_transform(df):
    dic = {1:np.random.uniform(0,4), 2:np.random.uniform(4,6), 3:np.random.uniform(6,8), 4:np.random.uniform(8,10), 5:np.random.uniform(10,12), 6:np.random.uniform(12,14)}
    if df['op_time'] < 7:
        return dic[df['op_time']]
    else:
        return df['op_time1']
    
# travel distance function
def travel_dist_transform(df):
    dic = {1:np.random.uniform(0,50), 2:np.random.uniform(50,100), 3:np.random.uniform(100,130), 4:np.random.uniform(130,160), 5:np.random.uniform(160,190), 6:np.random.uniform(190,220), 7:np.random.uniform(22,250), 8:np.random.uniform(250,280)}
    return dic[df['travel_distance']]

# no passenger time function
def no_passenger_time_transform(df):
    dic = {1:np.random.uniform(0,0.5), 2:np.random.uniform(0.5,1), 3:np.random.uniform(1,2), 4:np.random.uniform(2,3), 5:np.random.uniform(3,4), 6:np.random.uniform(4,5), 7:np.random.uniform(5,6), 8:np.random.uniform(6,7), 9:np.random.uniform(7,8), 10:np.random.uniform(8,9), 11:np.random.uniform(9,10), 12:np.random.uniform(10,24)}
    return dic[df['no_passenger_time']]

# daily trip function
def trip_transform(df):
    dic = {1: np.random.random_integers(1,3), 2:np.random.random_integers(4,6), 3:np.random.random_integers(7,9), 4:np.random.random_integers(10,12), 5:np.random.random_integers(13,15), 6:np.random.random_integers(16,18), 7:np.random.random_integers(19,21), 8:np.random.random_integers(22,24), 9:np.random.random_integers(25,27), 10:np.random.random_integers(28,30), 11:np.random.random_integers(35,40)}
    return dic[df['trip']]

# transform off day and operation duration
tw_taxi['off_day'] = tw_taxi.apply(off_day_transform, axis=1)
tw_taxi['op_time'] = tw_taxi.apply(op_time_transform, axis=1)
tw_taxi['travel_distance'] = tw_taxi['travel_distance'].fillna(np.random.random_integers(1, 8))
tw_taxi['travel_distance'] = tw_taxi.apply(travel_dist_transform, axis=1)
tw_taxi['no_passenger_time'] = tw_taxi['no_passenger_time'].fillna(np.random.random_integers(1, 12))
tw_taxi['no_passenger_time'] = tw_taxi.apply(no_passenger_time_transform, axis=1)
tw_taxi['trip'] = tw_taxi['trip'].fillna(np.random.random_integers(1, 11))
tw_taxi['trip'] = tw_taxi.apply(trip_transform, axis=1)
tw_taxi = tw_taxi.drop(columns=['off_day1', 'op_time1', 'full_time', 'single_driver', 'city', 'city1'], axis=1)
tw_taxi = tw_taxi.fillna(0)
tw_taxi

/var/folders/91/7rfwkjr118d8hfgspxh0gly944z1vk/T/ipykernel_37682/2649683545.py:3: DeprecationWarning: This function is deprecated. Please call randint(1, 2 + 1) instead
  dic = {1: 0, 2:np.random.random_integers(1,2), 3:np.random.random_integers(3,4), 4:np.random.random_integers(5,6), 5:np.random.random_integers(7,8)}
/var/folders/91/7rfwkjr118d8hfgspxh0gly944z1vk/T/ipykernel_37682/2649683545.py:3: DeprecationWarning: This function is deprecated. Please call randint(3, 4 + 1) instead
  dic = {1: 0, 2:np.random.random_integers(1,2), 3:np.random.random_integers(3,4), 4:np.random.random_integers(5,6), 5:np.random.random_integers(7,8)}
/var/folders/91/7rfwkjr118d8hfgspxh0gly944z1vk/T/ipykernel_37682/2649683545.py:3: DeprecationWarning: This function is deprecated. Please call randint(5, 6 + 1) instead
  dic = {1: 0, 2:np.random.random_integers(1,2), 3:np.random.random_integers(3,4), 4:np.random.random_integers(5,6), 5:np.random.random_integers(7,8)}
/var/folders/91/7rfwkjr118d8hfgspxh0gly9

,travel_distance,off_day,slot1,slot2,slot3,slot4,slot5,slot6,slot7,slot8,slot9,slot10,slot11,op_time,no_passenger_time,trip
0,93.815022,5.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,8.129733,0.718056,5
1,105.987976,3.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,7.138860,1.651609,4
2,46.213796,8.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.869891,0.801577,6
4,107.369533,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,8.765754,1.925944,17
5,156.558536,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,10.566015,1.834809,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8850,134.532416,4.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,11.530591,3.461981,17
8855,204.549740,8.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,9.258298,0.254180,25
8856,170.711832,2.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,11.183736,0.608411,13
8857,81.867704,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,9.595808,4.953398,10


In [5]:
def total_slot(df):
    count = 0
    for i in range(1,12):
        count += df.iloc[1+i]
    return count

def trip_distribution(df):
    distribute = []
    _max = df['slot_count']
    _num = df['trip']
    if _max < 2:
        numbers = [_num]
    else:
        numbers = sorted(np.random.choice(int(_num),int(_max-1)))
    count = 0
    for i in range(1,12):
        if df['slot'+str(i)] == 0:
            distribute.append(0)
        else:
            if count == 0:
                distribute.append(numbers[0])
                count += 1
            elif count < _max-1:
                distribute.append(numbers[count] - numbers[count-1])
                count += 1
            else:
                distribute.append(int(_num - numbers[-1])) 
    # for i in range(1, int(_max-1)):
    #     distribute.append(numbers[i] - numbers[i-1])
    # distribute.append(int(_num - numbers[-1]))  
    return distribute 

tw_taxi['slot_count'] = tw_taxi.apply(total_slot, axis=1)
tw_taxi = tw_taxi.drop(tw_taxi[(tw_taxi['slot_count'] == 0) & (tw_taxi['trip'] > 0)].index)

tw_taxi['trip_distribution'] = tw_taxi.apply(trip_distribution, axis=1)
tw_taxi = tw_taxi.reset_index()
tw_taxi

,index,travel_distance,off_day,slot1,slot2,slot3,slot4,slot5,slot6,slot7,slot8,slot9,slot10,slot11,op_time,no_passenger_time,trip,slot_count,trip_distribution
0,0,93.815022,5.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,8.129733,0.718056,5,7.0,"[0, 0, 0, 0, 0, 0, 1, 2, 1, 0, 1]"
1,1,105.987976,3.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,7.138860,1.651609,4,3.0,"[0, 0, 1, 0, 2, 0, 1, 0, 0, 0, 0]"
2,2,46.213796,8.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.869891,0.801577,6,1.0,"[0, 0, 6.0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,4,107.369533,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,8.765754,1.925944,17,2.0,"[0, 0, 0, 0, 0, 0, 0, 16, 0, 1, 0]"
4,5,156.558536,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,10.566015,1.834809,15,2.0,"[0, 0, 0, 6, 0, 0, 0, 0, 9, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3735,8850,134.532416,4.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,11.530591,3.461981,17,7.0,"[0, 0, 0, 0, 0, 3, 6, 0, 1, 1, 6]"
3736,8855,204.549740,8.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,9.258298,0.254180,25,5.0,"[0, 0, 6, 8, 4, 6, 1, 0, 0, 0, 0]"
3737,8856,170.711832,2.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,11.183736,0.608411,13,4.0,"[6, 2, 3, 0, 0, 0, 0, 0, 0, 0, 2]"
3738,8857,81.867704,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,9.595808,4.953398,10,4.0,"[0, 0, 0, 0, 0, 0, 0, 0, 4, 1, 5]"


Up to now data information:

number of drivers: 3740
number of trip: 49469

市區定義：中正 大安 松山 信義 板橋 永和 中和 中山 (60% start points, # = 2244)

一級郊區定義： 文山 新店 蘆洲 三重 新莊 萬華 南港 內湖 大同 林口 士林 (30% start points, # = 1122)

郊區定義：ELSE (10% start points, # = 374)

除外：烏來 坪林 石碇 雙溪 貢寮

<!-- ![<District Level>](./level.png = 250x250) -->
<img src="./district_level.png" width="500">


In [6]:
# trajectories creation
start_point_col = ['id', 'lon', 'lat']
downtown_start_point = pd.read_csv('./start_point/DownTown.csv', usecols=['id', 'x', 'y'])
uptown_start_point = pd.read_csv('./start_point/UpTown.csv', usecols=['id', 'x', 'y'])
country_start_point = pd.read_csv('./start_point/Country.csv', usecols=['id', 'x', 'y'])
downtown_start_point.columns = start_point_col
uptown_start_point.columns = start_point_col
country_start_point.columns = start_point_col

downtown_start_point = downtown_start_point.sample(frac=1).reset_index(drop=True).drop(columns=['id'])
uptown_start_point = uptown_start_point.sample(frac=1).reset_index(drop=True).drop(columns=['id'])
country_start_point = country_start_point.sample(frac=1).reset_index(drop=True).drop(columns=['id'])

print(downtown_start_point.head())
print(uptown_start_point.head())
print(country_start_point.head())


          lon        lat
0  121.549345  25.014277
1  121.530996  25.084852
2  121.544927  25.053154
3  121.498015  24.989543
4  121.490166  25.002164
          lon        lat
0  121.605488  25.039811
1  121.501697  24.906872
2  121.503471  25.026539
3  121.578313  25.179203
4  121.577281  25.123735
          lon        lat
0  121.456436  24.943259
1  121.462495  24.890437
2  121.614106  25.224528
3  121.626458  25.054297
4  121.338932  24.940460


In [7]:
# # randomly concat start points
# all_start_point = pd.concat([downtown_start_point, uptown_start_point, country_start_point], axis=0, ignore_index=True)
# all_start_point = all_start_point.sample(frac=1).reset_index(drop=True)
# all_start_point = all_start_point.drop(columns=['id'], axis=1)
# print(all_start_point)
# taxi_start = pd.concat([tw_taxi, all_start_point], axis=1)
# taxi_start

In [8]:
sorted_tw_taxi = tw_taxi.sort_values(by=['trip'])
print(sorted_tw_taxi.shape[0])
country_tw_taxi = sorted_tw_taxi.iloc[:374, :]
uptown_tw_taxi = sorted_tw_taxi.iloc[374:1496, :]
downtown_tw_taxi = sorted_tw_taxi.iloc[1496:, :]
print(downtown_tw_taxi.shape[0])
print(uptown_tw_taxi.shape[0])
print(country_tw_taxi.shape[0])

3740
2244
1122
374


In [9]:
def traj_gen(df1, df2):
    def traj_gen_per_day(coordinate, trip_count, _start_time):
        # general settings
        avg_v = 15 # m/s, agv speed
        r = 50 # meter, pie radius, define how close is "near spot"
        theta = np.pi/12 #radian, pie angle
        vec_list = []
        for i in range(len(coordinate)):
            ang = np.random.uniform(0, 2*np.pi)
            vec = [math.cos(ang), math.sin(ang)]
            vec_list.append(vec)
        generator = GPTG(avg_v, r, theta, coordinate, vec_list, check_boundary=True)
        generator.create_trajectory_by_count(count=trip_count, start_time=_start_time)
        if len(coordinate) == 374:
            generator.output('./trajectory/country_'+_start_time[0].strftime("%Y-%m-%d")+'.csv', 'csv')
            # generator.output('./trajectory/country_'+_start_time[0].strftime("%Y-%m-%d")+'.geojson', 'geojson')
        elif len(coordinate) == 1122:
            generator.output('./trajectory/uptown_'+_start_time[0].strftime("%Y-%m-%d")+'.csv', 'csv')
            # generator.output('./trajectory/uptown_'+_start_time[0].strftime("%Y-%m-%d")+'.geojson', 'geojson')
        elif len(coordinate) == 2244:
            generator.output('./trajectory/downtown_'+_start_time[0].strftime("%Y-%m-%d")+'.csv', 'csv')
            # generator.output('./trajectory/downtown_'+_start_time[0].strftime("%Y-%m-%d")+'.geojson', 'geojson')
        
    start_date = datetime.datetime(2022,1,1,0,0,0)
    hr_slot = [0,4,6,8,10,12,14,16,18,20,22,24]
    start_hr = [np.nonzero(x)[0][0]  for x in df1['trip_distribution'].tolist()]
    end_hr =   [np.nonzero(x)[0][-1] for x in df1['trip_distribution'].tolist()]
    day_count = 31
    trip_list = df1['trip'].tolist()
    trip_list = [i*55 - 5 for i in trip_list]
    for day in range(day_count):
        exact_start_date = start_date + datetime.timedelta(days=day)
        start_timestamp = [exact_start_date + datetime.timedelta(hours=np.random.uniform(hr_slot[i],  hr_slot[i+1])) for i in start_hr]
        start_point = df2.sample(frac=1).values.tolist()
        traj_gen_per_day(start_point, trip_list, start_timestamp)
        print('current date: ' + exact_start_date.strftime("%Y-%m-%d"))
        
        
        

In [10]:
country_traj = traj_gen(country_tw_taxi, country_start_point)
country_traj

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
# uptown_traj = traj_gen(uptown_tw_taxi, uptown_start_point)
# uptown_traj

traj 0 completed
traj 1 completed
traj 2 completed
traj 3 completed
traj 4 completed
traj 5 completed
traj 6 completed
traj 7 completed
traj 8 completed
traj 9 completed
traj 10 completed
traj 11 completed
traj 12 completed
traj 13 completed
traj 14 completed
traj 15 completed
traj 16 completed
traj 17 completed
traj 18 completed
traj 19 completed
traj 20 completed
traj 21 completed
traj 22 completed
traj 23 completed
traj 24 completed
traj 25 completed
traj 26 completed
traj 27 completed
traj 28 completed
traj 29 completed
traj 30 completed
traj 31 completed
traj 32 completed
traj 33 completed
traj 34 completed
traj 35 completed
traj 36 completed
traj 37 completed
traj 38 completed
traj 39 completed
traj 40 completed
traj 41 completed
traj 42 completed
traj 43 completed
traj 44 completed
traj 45 completed
traj 46 completed
traj 47 completed
traj 48 completed
traj 49 completed
traj 50 completed
traj 51 completed
traj 52 completed
traj 53 completed
traj 54 completed
traj 55 completed
tr

KeyboardInterrupt: 

In [ ]:
# downtown_traj = traj_gen(downtown_tw_taxi, downtown_start_point)
# downtown_traj

In [ ]:
# import glob
# country_traj_list = []
# country_file = glob.glob('./country_*.csv')
# for file in country_file:
#     df = pd.read_csv(file, usecols=['lon', 'lat', 'timestamp'])
#     country_traj_list.append(df)
    
# all_country_traj = pd.concat(country_traj_list, axis=0, ignore_index=True)
